In [ ]:
import random
from ipycanvas import MultiCanvas
from ipywidgets import Output, Button, VBox, Image

# Bilder laden
welcome_img = Image.from_file('99_welcome.png')
game_over_img = Image.from_file('98_game_over.png')
congratulations = Image.from_file('congrats.jpeg')

# Layout-Definition
layout = {'border': '3px solid black'}
mcanvas = MultiCanvas(3, width=400, height=300, layout=layout)
canvas1, canvas2, canvas3 = mcanvas
out = Output(layout={'border': '2px solid black'})

# Globale Variablen
# state = {'word_to_guess': '', 'guessed_letters': [], 'attempts': 6}

# dict fuer globale Variabeln / Lösungswort
state= {}

# Begrüssungsnachricht anzeigen
def draw_greeting():
    canvas1.draw_image(welcome_img, 0, 0, width=400, height=300)
    canvas1.font = '22px Arial'
    canvas1.fill_style = 'white'
    canvas1.fill_text('Wähle eine Kategorie um zu starten', 30, 270)

# Ihre Wortlisten und Auswahllogik
def choose_word(category):
    if category == 1:
        words = ["python", "jupyter", "notebook", "hangman", "programming", "challenge"]
    elif category == 2:
        words = ["deutschland", "frankreich", "italien", "liechtenstein", "niederlande", "norwegen",
                 "österreich", "portugal", "schweiz", "spanien"]
    elif category == 3:
        words = ["zürich", "winterthur", "schaffhausen", "frauenfeld",
                 "aarau", "baden", "davos", "bellinzona", "solothurn", "bern"]
    elif category == 4:
        words = ["elefant", "gorilla", "tiger", "panda", "krokodil",
                "gepard", "nashorn", "giraffe", "känguru", "pinguin"]
    return random.choice(words)

# Aktuellen Stand des Wortes anzeigen
def display_word():
    displayed = ""
    for letter in state['word_to_guess']:
        if letter in state['guessed_letters']:
            displayed += letter
        else:
            displayed += " _ "
    return displayed

# Hangman zeichnen basierend auf den verbleibenden Versuchen
def draw_hangman():
    attempts = state['attempts'] # Verwenden Sie das globale Statuswörterbuch
    canvas1.line_width = 6
    canvas1.stroke_line(50, 200, 150, 200)
    canvas1.stroke_line(100, 200, 100, 50)
    canvas1.stroke_line(100, 50, 150, 50)
    if attempts < 6:
        canvas1.fill_arc(145, 60, 10, 0, 2 * 3.14)
    if attempts < 5:
        canvas1.stroke_line(150, 70, 150, 120)
    if attempts < 4:
        canvas1.stroke_line(150, 80, 130, 100)
    if attempts < 3:
        canvas1.stroke_line(150, 80, 170, 100)
    if attempts < 2:
        canvas1.stroke_line(150, 120, 130, 140)
    if attempts < 1:
        canvas1.stroke_line(150, 120, 170, 140)

# Hangman und Wortstand aktualisieren
def update_hangman():
    canvas1.clear()
    draw_hangman()  # Zeichnet den aktuellen Hangman-Zustand
    canvas1.font = '22px Calibri'
    canvas1.fill_style = 'black'
    canvas1.fill_text("Aktueller Stand: " + display_word(), 30, 240)
    canvas1.fill_text("Versuche übrig: " + str(state['attempts']), 30, 275)

    if state['attempts'] == 0:
        canvas2.draw_image(game_over_img, 0, 0, width=400, height=300)
    else:
        current_display = display_word()
        if "_" not in current_display:  # Überprüfung auf Spielabschluss mit Bildanzeige -> Gewonnen
            canvas3.draw_image(congratulations, 0, 0, width=400, height=300)

# Button-Logik für Kategorienauswahl
def choose_category_buttons():
    action1_bt = Button(description='Programmierung', layout=layout)
    action2_bt = Button(description='Europäische Länder', layout=layout)
    action3_bt = Button(description='Schweizer Städte', layout=layout)
    action4_bt = Button(description='Tiere', layout=layout)

    action1_bt.on_click(lambda _: start_game(1))
    action2_bt.on_click(lambda _: start_game(2))
    action3_bt.on_click(lambda _: start_game(3))
    action4_bt.on_click(lambda _: start_game(4))
    # VBox zurueckgaben, wird in hangman dann dargestellt
    return VBox([action1_bt, action2_bt, action3_bt, action4_bt])

# Mit dieser Funktion wird das Spiel gestartet
def hangman():
    draw_greeting()
    display(out, mcanvas, choose_category_buttons())    
    attempts = 6

# Korrektur für die Verwendung von 'out.clear_output()'
@out.capture()
def start_game(category):
    out.clear_output()
    mcanvas.clear()
    state['word_to_guess'] = choose_word(category)
    state['guessed_letters'] = []
    state['attempts'] = 6
    hint_letters = random.sample(state['word_to_guess'], 2)
    for letter in hint_letters:
        if letter not in state['guessed_letters']:
            state['guessed_letters'].append(letter)
    update_hangman()
    print("Rate einen Buchstaben: ")

# Logik für Buchstabenraten und Aktualisierung des Spielzustandes
@out.capture()
def process_key(key, *flags):
    guess = key.lower()
    word_to_guess = state['word_to_guess']
    if guess in state['guessed_letters']:
        print("Du hast diesen Buchstaben bereits geraten. Versuche es erneut.")
        return
    state['guessed_letters'].append(guess)
    if guess not in word_to_guess:
        state['attempts'] -= 1
        print("Falsch geraten!")
    else:
        print("Richtig geraten!")  
    update_hangman()    
    if state['attempts'] == 0:
        print("Das richtige Wort war:", word_to_guess)
    # Rufen Sie display_word() auf, um den aktuellen Anzeigestatus des Wortes abzurufen
    current_display = display_word()
    if "_" not in current_display: # Überprüfen Sie die Vollständigkeit anhand der aktualisierten Anzeige
        print("Herzlichen Glückwunsch! Du hast das Wort gefunden:", word_to_guess)

# Callback für on_key_down
mcanvas.on_key_down(process_key)

# Spielinitialisierung
hangman()